In [16]:
!pip install transformers

### Import


In [17]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.metrics import accuracy_score, precision_score, recall_score


import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import torch
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse
%matplotlib inline
seed = 42
import os
os.environ['OMP_NUM_THREADS'] = '4'


In [18]:
# Importing necessary libraries and mounting Google Drive for data access
from google.colab import drive
drive.mount('/content/drive')

# Reading the training and testing data along with labels
train_data = pd.read_csv('/content/drive/MyDrive/ML_PROJECT/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ML_PROJECT/test.csv')
test_labels = pd.read_csv('/content/drive/MyDrive/ML_PROJECT/test_labels.csv')

# Merging test data with labels and filtering rows with label '-1'
test_merged = pd.merge(test_data, test_labels, on='id')
test_filtered = test_merged[(test_merged.iloc[:, 2:] != -1).all(axis=1)]

# Separating toxic and non-toxic comments in the training data
toxic_comments = train_data[train_data.iloc[:, 2:].sum(axis=1) > 0]
non_toxic_comments = train_data[train_data.iloc[:, 2:].sum(axis=1) == 0]

# Balancing the dataset to address class imbalance
if len(non_toxic_comments) > len(toxic_comments):
    non_toxic_sample = non_toxic_comments.sample(n=len(toxic_comments), random_state=42)
    balanced_data = pd.concat([toxic_comments, non_toxic_sample])
else:
    toxic_sample = toxic_comments.sample(n=len(non_toxic_comments), random_state=42)
    balanced_data = pd.concat([toxic_sample, non_toxic_comments])

# Shuffling the balanced dataset for randomness and resetting the index
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Splitting the balanced dataset into training, validation, and testing sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    balanced_data['comment_text'], balanced_data.iloc[:, 2:], test_size=0.25, random_state=42)

test_texts, val_texts, test_labels, val_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)


### Tokenization and Encoding using BERT for Toxic Comment Classification

1. **Tokenization Function:**
   - `tokenize_and_encode_bert` function is defined to tokenize and encode comments using the BERT tokenizer.
   - It leverages BERT's tokenization to convert comments into numerical representations suitable for model input.
   - BERT tokenization allows the model to understand contextual information and relationships between words.

2. **BERT Configuration:**
   - BERT tokenizer (`bert_tokenizer`) is loaded from 'bert-base-uncased' with case-insensitivity.
   - BERT model (`bert_model`) for sequence classification with six labels is loaded.
   - 'bert-base-uncased' is chosen for its balanced performance and efficiency in handling uncased text.

3. **Data Preparation:**
   - Training, testing, and validation datasets are tokenized and encoded using the `tokenize_and_encode_bert` function.
   - Input IDs, attention masks, and labels are converted to PyTorch datasets (`train_dataset`, `test_dataset`, `val_dataset`).
   - Data loaders (`train_loader`, `test_loader`, `val_loader`) are created for efficient batch processing.

4. **Optimization:**
   - AdamW optimizer is instantiated for fine-tuning BERT model parameters.
   - AdamW is chosen for its ability to handle weight decay in a way that suits transformer-based models like BERT.
   - The learning rate is set to 2e-5 to balance training speed and convergence.

This markdown summarizes the implementation of BERT-based tokenization and encoding for toxic comment classification. The chosen tokenizer and BERT configuration enhance the model's ability to understand language nuances, while AdamW optimizer is employed for effective fine-tuning during training.


In [20]:
# Tokenization and Encoding using BERT for Toxic Comment Classification
# Objective: Convert comments into numerical representations suitable for BERT model input.

def tokenize_and_encode_bert(tokenizer, comments, labels, max_length=128):
    # Lists to store tokenized input IDs and attention masks
    input_ids, attention_masks = [], []
    
    # Iterate through comments for tokenization
    for comment in comments:
        # Encode comment using BERT tokenizer
        encoded_dict = tokenizer.encode_plus(
            comment, add_special_tokens=True, max_length=max_length,
            pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
        
        # Append input IDs and attention masks
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Concatenate lists to create tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    # Convert labels to PyTorch tensor
    labels = torch.tensor(labels.values, dtype=torch.float32)

    # Return tokenized input IDs, attention masks, and labels
    return input_ids, attention_masks, labels


In [21]:
# BERT Tokenizer Initialization
# Objective: Load BERT tokenizer for tokenizing and encoding text data.

# Using BERT-base-uncased model with lowercase tokens
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [22]:
# BERT Model Initialization
# Objective: Load BERT model for sequence classification with specified configuration.

# Using BERT-base-uncased model with 6 output labels for toxic comment classification
# Moving the model to GPU if available, otherwise using CPU
bert_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=6).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Tokenization and Encoding for BERT
# Objective: Tokenize and encode comments using the BERT tokenizer, and prepare input tensors.

# Tokenizing and encoding training set comments
train_inputs, train_masks, train_labels = tokenize_and_encode_bert(
    bert_tokenizer, train_texts, train_labels)

# Tokenizing and encoding test set comments
test_inputs, test_masks, test_labels = tokenize_and_encode_bert(
    bert_tokenizer, test_texts, test_labels)

# Tokenizing and encoding validation set comments
val_inputs, val_masks, val_labels = tokenize_and_encode_bert(
    bert_tokenizer, val_texts, val_labels)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
# Creating PyTorch DataLoader for BERT
# Objective: Create DataLoader instances for training, testing, and validation sets.

# Creating training dataset
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)

# Creating test dataset
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Creating validation dataset
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

# Setting batch size
batch_size = 32

# Creating training DataLoader with random sampling
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)

# Creating test DataLoader with sequential sampling
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# Creating validation DataLoader with sequential sampling
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [25]:
optimizer = AdamW(bert_model.parameters(), lr=2e-5)


# Training and Evaluating BERT Model for Toxicity Classification

## Objective:
Train a BERT model to classify toxic comments into various categories using a specified DataLoader, optimizer, and device. Evaluate the model's performance on test data and make toxicity predictions for user input.

## Steps:

### 1. Training BERT Model:
- **Objective:** Train the BERT model on the training data.
- **Steps:**
  - Utilize a custom function `train_bert_model` to handle training with specified DataLoader, optimizer, and device.
  - Iterate through epochs, setting the model to training mode, calculating loss, and backpropagating.

### 2. Evaluating BERT Model:
- **Objective:** Evaluate the BERT model on the test data.
- **Steps:**
  - Utilize a custom function `evaluate_bert_model` to assess accuracy, precision, and recall.
  - Threshold predicted probabilities and calculate evaluation metrics using scikit-learn.

### 3. Predicting User Input:
- **Objective:** Make toxicity predictions for user input.
- **Steps:**
  - Utilize a custom function `predict_user_input` to tokenize, encode, and predict toxicity labels for user input.

## Components Used:

- **BERT Model:** Utilizing a pre-trained BERT model for sequence classification with six toxicity labels.
- **DataLoader:** Loading and iterating through the balanced training, testing, and validation datasets in batches.
- **Optimizer:** Implementing AdamW optimizer for model training.
- **Tokenizer:** Using the BERT tokenizer to encode comments for model input.
- **Device:** Training and prediction on GPU if available, else on CPU.

## Why Pretrained Encoder-Transformers like BERT:

- **Effectiveness:** Pretrained models like BERT capture contextual information, enhancing performance on complex tasks like toxicity classification.
- **Transfer Learning:** Leveraging knowledge from a large pretraining dataset accelerates learning on a smaller task-specific dataset.
- **Semantic Understanding:** BERT's contextual embeddings help discern nuances and contextual meanings in text, crucial for toxicity assessment.
- **Efficiency:** Pretrained models reduce the need for extensive training on task-specific data, saving computational resources.



In [26]:
# Training BERT Model
# Objective: Train the BERT model on the training data using the specified DataLoader, optimizer, and device.

def train_bert_model(model, train_loader, optimizer, device, num_epochs=3):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_val_loss = 0, 0

        # Training
        for batch in train_loader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                batch = tuple(t.to(device) for t in batch)
                inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
                outputs = model(**inputs)
                loss = outputs.loss
                total_val_loss += loss.item()

        # Print epoch-wise training and validation losses
        print(f"Epoch {epoch + 1}/{num_epochs} - Training Loss: {total_loss/len(train_loader)}, Validation Loss: {total_val_loss/len(val_loader)}")


# Training BERT model using defined function
train_bert_model(bert_model, train_loader, optimizer, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Epoch 1/3 - Training Loss: 0.1982150400808857, Validation Loss: 0.15240689619319644
Epoch 2/3 - Training Loss: 0.1365145442960845, Validation Loss: 0.14957816452961267
Epoch 3/3 - Training Loss: 0.11254309388991254, Validation Loss: 0.15509002415214


In [27]:
# train_bert_model(bert_model, train_loader, optimizer, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


In [28]:
# Evaluating BERT Model
# Objective: Evaluate the BERT model on the test data using the specified DataLoader and device.

def evaluate_bert_model(model, test_loader, device):
    model.eval()
    true_labels, predicted_probs = [], []

    # Testing
    with torch.no_grad():
        for batch in test_loader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
            outputs = model(**inputs)
            logits = torch.sigmoid(outputs.logits)
            predicted_probs.extend(logits.cpu().numpy())
            true_labels.extend(batch[2].cpu().numpy())

    # Thresholding predicted probabilities
    predicted_labels = (np.array(predicted_probs) > 0.5).astype(int)

    # Calculating evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')

    # Print evaluation metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')


# Evaluating BERT model using defined function
evaluate_bert_model(bert_model, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Accuracy: 0.7133
Precision: 0.8083
Recall: 0.8590


In [31]:
# Save the tokenizer and model in the same directory
output_dir = "Saved_model"
# Save model's state dictionary and configuration
bert_model.save_pretrained(output_dir)
# Save tokenizer's configuration and vocabulary
bert_tokenizer.save_pretrained(output_dir)


('Saved_model/tokenizer_config.json',
 'Saved_model/special_tokens_map.json',
 'Saved_model/vocab.txt',
 'Saved_model/added_tokens.json')

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "Saved_model"
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
Bert_Model = BertForSequenceClassification.from_pretrained(
    model_name).to(device)

In [39]:
# Predicting User Input
# Objective: Make toxicity predictions for user input using the BERT model, tokenizer, and specified device.

def predict_user_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer, device=device):
    # Tokenize and encode user input
    user_input = [input_text]
    user_encodings = tokenizer(
        user_input, truncation=True, padding=True, return_tensors="pt")
    user_dataset = TensorDataset(
        user_encodings['input_ids'], user_encodings['attention_mask'])
    user_loader = DataLoader(user_dataset, batch_size=1, shuffle=False)

    # Model prediction
    model.eval()
    with torch.no_grad():
        for batch in user_loader:
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)

    # Thresholding predicted probabilities
    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)

    # Creating a dictionary with predicted labels
    labels_list = ['toxic', 'severe_toxic', 'obscene',
                   'threat', 'insult', 'identity_hate']
    result = dict(zip(labels_list, predicted_labels[0]))

    return result


# Example: Predicting toxicity for a user input
text = 'Hi, how are you?'
predict_user_input(input_text=text)


{'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0}

In [40]:
text = 'Are you stupid?'
predict_user_input(input_text=text)

{'toxic': 1,
 'severe_toxic': 0,
 'obscene': 1,
 'threat': 0,
 'insult': 1,
 'identity_hate': 0}

In [41]:
text = 'Machine learning is hard'
predict_user_input(input_text=text)

{'toxic': 0,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 0,
 'identity_hate': 0}

In [42]:
text = 'You are Dumb!'
predict_user_input(input_text=text)

{'toxic': 1,
 'severe_toxic': 0,
 'obscene': 0,
 'threat': 0,
 'insult': 1,
 'identity_hate': 0}

In [48]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def print_classification_metrics(true_labels, predicted_labels):
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for i, class_name in enumerate(class_names):
        print(f"Metrics for class: {class_name}")
        print("Confusion Matrix:")
        print(confusion_matrix(true_labels[:, i], predicted_labels[:, i]))
        print("Precision: {:.2f}".format(precision_score(true_labels[:, i], predicted_labels[:, i])))
        print("Recall: {:.2f}".format(recall_score(true_labels[:, i], predicted_labels[:, i])))
        print("F1-Score: {:.2f}".format(f1_score(true_labels[:, i], predicted_labels[:, i])))
        print("-" * 30)

In [47]:
def evaluate_bert_model_for_classification(model, test_loader, device):
    model.eval()
    true_labels, predicted_probs = [], []
    with torch.no_grad():
        for batch in test_loader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
            outputs = model(**inputs)
            logits = torch.sigmoid(outputs.logits)
            predicted_probs.extend(logits.cpu().numpy())
            true_labels.extend(batch[2].cpu().numpy())

    predicted_labels = (np.array(predicted_probs) > 0.5).astype(int)
    return np.array(true_labels), predicted_labels

# Call the modified evaluation function and get true and predicted labels
true_labels, predicted_labels = evaluate_bert_model_for_classification(bert_model, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Now call the print_classification_metrics function with the true and predicted labels
print_classification_metrics(true_labels, predicted_labels)


Metrics for class: toxic
Confusion Matrix:
[[1940  208]
 [ 109 1799]]
Precision: 0.90
Recall: 0.94
F1-Score: 0.92
------------------------------
Metrics for class: severe_toxic
Confusion Matrix:
[[3757   95]
 [ 105   99]]
Precision: 0.51
Recall: 0.49
F1-Score: 0.50
------------------------------
Metrics for class: obscene
Confusion Matrix:
[[2802  222]
 [ 123  909]]
Precision: 0.80
Recall: 0.88
F1-Score: 0.84
------------------------------
Metrics for class: threat
Confusion Matrix:
[[3961   32]
 [  16   47]]
Precision: 0.59
Recall: 0.75
F1-Score: 0.66
------------------------------
Metrics for class: insult
Confusion Matrix:
[[2832  233]
 [ 218  773]]
Precision: 0.77
Recall: 0.78
F1-Score: 0.77
------------------------------
Metrics for class: identity_hate
Confusion Matrix:
[[3792   99]
 [  44  121]]
Precision: 0.55
Recall: 0.73
F1-Score: 0.63
------------------------------


In [49]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

def print_classification_metrics_invidiual_classes(true_labels, predicted_labels):
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for i, class_name in enumerate(class_names):
        print(f"Metrics for class: {class_name}")

        # Calculating metrics for class 0
        precision_0 = precision_score(true_labels[:, i], predicted_labels[:, i], pos_label=0)
        recall_0 = recall_score(true_labels[:, i], predicted_labels[:, i], pos_label=0)
        f1_0 = f1_score(true_labels[:, i], predicted_labels[:, i], pos_label=0)

        # Calculating metrics for class 1
        precision_1 = precision_score(true_labels[:, i], predicted_labels[:, i], pos_label=1)
        recall_1 = recall_score(true_labels[:, i], predicted_labels[:, i], pos_label=1)
        f1_1 = f1_score(true_labels[:, i], predicted_labels[:, i], pos_label=1)

        # Printing metrics
        print(f"Class 0: Precision: {precision_0:.2f}, Recall: {recall_0:.2f}, F1-Score: {f1_0:.2f}")
        print(f"Class 1: Precision: {precision_1:.2f}, Recall: {recall_1:.2f}, F1-Score: {f1_1:.2f}")

        # Confusion matrix
        cm = confusion_matrix(true_labels[:, i], predicted_labels[:, i])
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

# Call this function with the true and predicted labels
print_classification_metrics_invidiual_classes(true_labels, predicted_labels)


Metrics for class: toxic
Class 0: Precision: 0.95, Recall: 0.90, F1-Score: 0.92
Class 1: Precision: 0.90, Recall: 0.94, F1-Score: 0.92
Confusion Matrix:
[[1940  208]
 [ 109 1799]]
------------------------------
Metrics for class: severe_toxic
Class 0: Precision: 0.97, Recall: 0.98, F1-Score: 0.97
Class 1: Precision: 0.51, Recall: 0.49, F1-Score: 0.50
Confusion Matrix:
[[3757   95]
 [ 105   99]]
------------------------------
Metrics for class: obscene
Class 0: Precision: 0.96, Recall: 0.93, F1-Score: 0.94
Class 1: Precision: 0.80, Recall: 0.88, F1-Score: 0.84
Confusion Matrix:
[[2802  222]
 [ 123  909]]
------------------------------
Metrics for class: threat
Class 0: Precision: 1.00, Recall: 0.99, F1-Score: 0.99
Class 1: Precision: 0.59, Recall: 0.75, F1-Score: 0.66
Confusion Matrix:
[[3961   32]
 [  16   47]]
------------------------------
Metrics for class: insult
Class 0: Precision: 0.93, Recall: 0.92, F1-Score: 0.93
Class 1: Precision: 0.77, Recall: 0.78, F1-Score: 0.77
Confusion 

## Findings

### Toxic:
The model achieves robust performance in distinguishing toxic and non-toxic comments, yielding an overall accuracy of 92%. Precision for non-toxic comments is high (0.95), with a balanced recall of 0.90. For toxic comments, precision remains strong at 0.90, with an improved recall of 0.94, resulting in an impressive F1-Score of 0.92.

### Severe Toxic:
The model exhibits excellent accuracy, reaching 97%, in classifying severe toxic and non-severe toxic comments. Precision for non-severe toxic comments is exceptionally high (0.97), coupled with a recall of 0.98. However, for severe toxic comments, there is a trade-off with precision dropping to 0.51, and recall at 0.49, resulting in a moderate F1-Score of 0.50.

### Obscene:
The model performs admirably in distinguishing obscene and non-obscene comments, achieving an accuracy of 94%. Precision for non-obscene comments is notably high (0.96), with a balanced recall of 0.93. For obscene comments, precision is commendable at 0.80, and recall improves to 0.88, resulting in a robust F1-Score of 0.84.

### Threat:
With an outstanding accuracy of 99%, the model excels in identifying non-threatening and threatening comments. Precision for non-threatening comments is perfect (1.00), accompanied by an impressive recall of 0.99. However, for threatening comments, precision is moderate (0.59), and recall is 0.75, yielding a balanced F1-Score of 0.66.

In summary, the model demonstrates consistent and effective performance across various classes, with specific precision-recall trade-offs in severe toxic and threatening comments. Achieving a balance between precision and recall remains crucial for different application scenarios.


## Conclusion

It is clear that Pre Encoded transfomer using BERT performed the best out of the lot, producing high accuracy and relatively satisfying F1, precision, and recall scores, compared to other models. This is unsurpsing, as Transformers are the state of the art in Natural language procesing. Indeed it performed brilliantly well in custom trained sentences we formed and put into.

While improvements could follow better data pre processing of enabling URLS etc, and maybe perhaps better mutli class data sampling, the results of this model on its own suggest a good trained model. BERT performs the best out of all the three models, in almost all cases. Hence, Model 3> Model 2> Model 1 follows, BERT performing the best, and OUR state of art model.

This is all from our side, Thank you for reading :)

The model was heavily complex wo we had to do multiple readings, one of the main sources we were inspired by  was:
https://www.geeksforgeeks.org/toxic-comment-classification-using-bert/